In [1]:
!pip install llama-index-core
!pip install llama-index-llms-ollama
!pip install llama-index-embeddings-ollama
!pip install llama-index-utils-workflow
!pip install llama-parse

If your environment does not have `wget` , make sure to install that as well.

In [1]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

In [3]:
import os, getpass


# def _set_env(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass.getpass(f"{var}: ")

# #llx-k2rELGB1DcrDZM9CXnLaubaJ2Oo6hNxFaj0uK422qeit00te
# _set_env("LLAMA_CLOUD_API_KEY")

In [2]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import Settings

# Connect to Ollama running in Docker (exposed on port 11434)
llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434")

messages = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="You are a helpful assistant that answers in one sentence.",
    ),
    ChatMessage(
        role=MessageRole.USER,
        content="Xin chào, hãy giới thiệu về bạn",
    ),
]

response = llm.chat(messages)
print(response)

# Configure settings to use Ollama
Settings.llm = Ollama(model="llama3.2:latest", base_url="http://localhost:11434", request_timeout=3000)
Settings.embed_model = OllamaEmbedding(model_name='mxbai-embed-large:335m', base_url="http://localhost:11434", request_timeout=3000)


assistant: Tôi là một trợ lý ảo thông minh được thiết kế để cung cấp thông tin chính xác và hữu ích cho người dùng với nhiều ngôn ngữ hỗ trợ khác nhau.


In [3]:
from qdrant_client import QdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex

# 1. Kết nối tới Qdrant
qdrant_client = QdrantClient("localhost", port=6333)
collection_name = "llamaindex_collection"

In [6]:
# Kiểm tra collection
collections = qdrant_client.get_collections()
if collection_name in [col.name for col in collections.collections]:
    print(f"Collection '{collection_name}' đã tồn tại.")
else:
    print(f"Collection '{collection_name}' chưa tồn tại. Cần tạo mới trước.")

Collection 'llamaindex_collection' đã tồn tại.


In [7]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,)

# Tạo Qdrant Vector Store
vector_store = QdrantVectorStore(client=qdrant_client, collection_name=collection_name)

# Tạo Storage Context với Qdrant
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 2. Load tài liệu từ thư mục "data"
documents = SimpleDirectoryReader("./law/Luật").load_data()

index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
)

In [8]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex

collection_name = "llamaindex_collection"

# 2. Tạo Qdrant Vector Store từ collection đã tồn tại
vector_store = QdrantVectorStore(
    client = qdrant_client,
    # aclient=async_client, 
    collection_name=collection_name
)

# 3. Tạo VectorStoreIndex từ Vector Store
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

# (Tùy chọn) Kiểm tra hoặc sử dụng index
query_engine = index.as_query_engine()

In [9]:
# Kiểm tra cấu hình collection
info = qdrant_client.get_collection(collection_name="llamaindex_collection")
print(info.config.params.vectors)  # In ra kích thước vector

size=1024 distance=<Distance.COSINE: 'Cosine'> hnsw_config=None quantization_config=None on_disk=None datatype=None multivector_config=None


In [10]:
query_engine = index.as_query_engine()
response = query_engine.query("Thông tin nào quan trọng nhất trong tài liệu?")
print(response)

Thông tin thị trường lao động gắn với tên, địa chỉ cụ thể của từng tổ chức, cá nhân.


In [11]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='4c7622c1-8539-4844-9e08-a43539b80467', embedding=None, metadata={'page_label': '7', 'file_name': 'Luật việc làm 2013.pdf', 'file_path': '/home/tuandatebayo/WorkSpace/law/Luật/Luật việc làm 2013.pdf', 'file_type': 'application/pdf', 'file_size': 215616, 'creation_date': '2025-03-25', 'last_modified_date': '2025-03-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='76cd9667-af94-4071-9e4d-39b0f897f37d', node_type='4', metadata={'page_label': '7', 'file_name': 'Luật việc làm 2013.pdf', 'file_path': '/home/tuandatebayo/WorkSpace/law/Luật/Luật việc làm 2013.pdf', 'file_type': 'application/pdf', 'file_size': 215616, 'creation_date': '2025-03-25', 'last_modified_date': '2025-03

In [4]:
from qdrant_client import AsyncQdrantClient

# Kết nối với Qdrant
async_client = AsyncQdrantClient("localhost", port=6333)

async def get_collection_info():
    collection_info = await async_client.get_collection("llamaindex_collection")
    print(collection_info)

# Run the async function
import asyncio
asyncio.run(get_collection_info())

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=1566 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=No

In [9]:
# Lấy danh sách collections
collections = async_client.get_collections()
print("Collections hiện có:", collections)

Collections hiện có: <coroutine object AsyncQdrantClient.get_collections at 0x7c22e0c19e40>


/tmp/ipykernel_96113/2981485355.py:2: RuntimeWarning: coroutine 'AsyncQdrantClient.get_collections' was never awaited
  collections = async_client.get_collections()


In [8]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex

collection_name = "llamaindex_collection"

# 2. Tạo Qdrant Vector Store từ collection đã tồn tại
vector_store = QdrantVectorStore(
    aclient=async_client, 
    collection_name=collection_name
)

# 3. Tạo VectorStoreIndex từ Vector Store
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

# (Tùy chọn) Kiểm tra hoặc sử dụng index
query_engine = index.as_query_engine()
# response = query_engine.query("What was San Francisco's budget for Police Department in 2023?")
# print(response)

In [10]:
query_engine = index.as_query_engine(similarity_top_k=3)
response = query_engine.query(
    "Hãy nêu nội dung của Điều 16. Nội dung hoạt động của sàn giao dịch bất động sản"
)
print(response.source_nodes)
print(response)

#can check result at page 67

AttributeError: 'NoneType' object has no attribute 'search'

In [11]:
from llama_index.core.tools import QueryEngineTool

law_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="labor_law_tool", # <--- Changed name
    description="Công cụ truy vấn thông tin chi tiết về các quy định pháp luật lao động tại Việt Nam, bao gồm Bộ luật Lao động, Luật Việc làm, Luật An toàn vệ sinh lao động, và các văn bản liên quan.", # <--- Updated description
)


# budget_tool = QueryEngineTool.from_defaults(
#     query_engine,
#     name="san_francisco_budget_2023",
#     description="A RAG engine with extremely detailed information about the 2023 San Francisco budget.",
# )

Đề tài: “Ứng dụng Multi-Agent để kiểm tra hợp đồng pháp lý hỗ trợ sinh viên luật”
Đối tượng người dùng:
Sinh viên ngành Luật: Đặc biệt là những người đang học các môn thực hành pháp lý, làm bài tập lớn hoặc thực tập liên quan đến phân tích, kiểm tra và hoàn thiện hợp đồng.
Vấn đề giải quyết:
Khó khăn trong việc phát hiện lỗi pháp lý trong hợp đồng: Sinh viên luật thường thiếu kinh nghiệm để nhận diện các sai sót hoặc thiếu sót trong hợp đồng (ví dụ: điều khoản mâu thuẫn, không tuân thủ luật, thiếu rõ ràng).
Hạn chế trong việc đánh giá tính khả thi của hợp đồng: Sinh viên gặp khó khăn khi xác định liệu hợp đồng có phù hợp với quy định pháp luật hiện hành và tình huống thực tế hay không.
Tốn thời gian kiểm tra thủ công: Việc đối chiếu hợp đồng với các văn bản luật, nghị định liên quan đòi hỏi nhiều công sức và dễ xảy ra sai sót do thiếu công cụ hỗ trợ chuyên sâu.
Cải tiến so với các công cụ hiện có:
Hệ thống đa tác tử (Multi-Agent): Không giống các công cụ tra cứu luật đơn thuần (như Vietlaw, Lawsoft), ứng dụng này sử dụng nhiều tác tử thông minh phối hợp để kiểm tra hợp đồng từ nhiều góc độ (pháp lý, ngữ nghĩa, logic).
Phân tích toàn diện: Thay vì chỉ tìm kiếm từ khóa hoặc cung cấp thông tin thô, hệ thống tự động phát hiện lỗi, đề xuất chỉnh sửa và giải thích lý do, hỗ trợ sinh viên học tập hiệu quả hơn.
Tính tương tác cao: Sinh viên có thể đặt câu hỏi trực tiếp (ví dụ: “Điều khoản này có hợp pháp không?”), và hệ thống sẽ trả lời dựa trên phân tích của các tác tử.
Đầu vào:
Dữ liệu đầu vào:
Bản nháp hợp đồng do sinh viên cung cấp (PDF, Word) – ví dụ: hợp đồng thuê nhà, mua bán tài sản.
Yêu cầu kiểm tra cụ thể (ví dụ: “Kiểm tra hợp đồng thuê nhà theo luật Việt Nam, giá 5 triệu/tháng, thời hạn 1 năm”).
(Tùy chọn) Tài liệu pháp lý liên quan (luật, nghị định, hợp đồng mẫu) hoặc tích hợp nguồn công khai như cổng thông tin pháp luật Việt Nam.
Đầu ra:
Bản hợp đồng đã kiểm tra và chỉnh sửa:
Các điều khoản được đánh dấu lỗi (nếu có) kèm đề xuất sửa đổi.
Trích dẫn nguồn luật liên quan (ví dụ: Điều 121 Luật Dân sự 2015).
Ghi chú giải thích chi tiết về các vấn đề pháp lý hoặc cải tiến (dành cho mục đích học tập).
Báo cáo phân tích hợp đồng:
Tổng hợp các lỗi pháp lý, mâu thuẫn nội dung hoặc rủi ro tiềm ẩn.
Đánh giá tính khả thi và hợp pháp của hợp đồng trong bối cảnh cụ thể.
Cách hệ thống hoạt động:
Hệ thống sử dụng nhiều tác tử (agent) thông minh phối hợp để kiểm tra hợp đồng một cách toàn diện:
Agent phân tích ngữ cảnh:
Đọc và hiểu nội dung hợp đồng, xác định loại hợp đồng (thuê nhà, mua bán, lao động, v.v.) và các yếu tố chính (giá, thời hạn, bên liên quan).
Đưa ra danh sách các yêu cầu pháp lý cần kiểm tra dựa trên ngữ cảnh.
Agent tra cứu pháp luật:
Thu thập dữ liệu từ tài liệu pháp lý đầu vào hoặc nguồn công khai (nếu được tích hợp).
Đối chiếu các điều khoản trong hợp đồng với quy định pháp luật hiện hành (ví dụ: Luật Dân sự, Luật Nhà ở).
Agent kiểm tra logic và ngữ nghĩa:
Phát hiện mâu thuẫn nội dung (ví dụ: điều khoản A yêu cầu thanh toán trước, nhưng điều khoản B lại nói thanh toán sau).
Đảm bảo ngôn ngữ rõ ràng, không mơ hồ, phù hợp với chuẩn pháp lý.
Agent đánh giá rủi ro:
Phân tích các rủi ro tiềm ẩn (ví dụ: thiếu điều khoản phạt vi phạm, không quy định rõ quyền chấm dứt hợp đồng).
Đề xuất bổ sung điều khoản để bảo vệ quyền lợi các bên.
Agent tổng hợp và xuất kết quả:
Kết hợp kết quả từ các tác tử trên để tạo bản hợp đồng đã chỉnh sửa và báo cáo phân tích.
Cung cấp giải thích chi tiết để sinh viên hiểu rõ từng thay đổi.
Ví dụ minh họa:
Đầu vào: Sinh viên tải lên hợp đồng thuê nhà với điều khoản “Bên thuê trả tiền thuê 5 triệu/tháng, không quy định thời hạn chấm dứt”.
Quá trình:
Agent ngữ cảnh: Xác định đây là hợp đồng thuê nhà, cần kiểm tra Luật Nhà ở 2014.
Agent tra cứu: Trích Điều 132 Luật Nhà ở 2014 về quyền chấm dứt hợp đồng.
Agent logic: Phát hiện thiếu quy định về thông báo chấm dứt hợp đồng.
Agent rủi ro: Đề xuất thêm điều khoản “Bên thuê phải báo trước 30 ngày nếu muốn chấm dứt”.
Đầu ra: Bản hợp đồng đã bổ sung điều khoản, kèm báo cáo giải thích.

Kết luận:
Ứng dụng Multi-Agent này không chỉ giúp sinh viên luật kiểm tra hợp đồng một cách nhanh chóng và chính xác mà còn đóng vai trò như một công cụ học tập, cung cấp kiến thức thực tiễn và giải thích chi tiết. So với các công cụ hiện có, hệ thống vượt trội nhờ khả năng phối hợp đa tác tử, phân tích ngữ nghĩa sâu và tự động hóa quy trình kiểm tra pháp lý.


In [ ]:
from typing import List, Union
from llama_index.core.workflow import (
    step,
    Event,
    Context,
    StartEvent,
    StopEvent,
    Workflow,
)
from llama_index.core.agent import FunctionCallingAgent
from collections import defaultdict
import json

# Định nghĩa các sự kiện
class OutlineEvent(Event):
    outline: str  # Kế hoạch kiểm tra hợp đồng

class QuestionEvent(Event):
    question: str  # Câu hỏi cụ thể về hợp đồng

class AnswerEvent(Event):
    question: str
    answer: str  # Câu trả lời kèm trích dẫn luật

class ReportEvent(Event):
    report: str  # Báo cáo phân tích hợp đồng

class ProgressEvent(Event):
    progress: str  # Tiến trình thực hiện

# Workflow kiểm tra hợp đồng lao động
class LaborContractReviewWorkflow(Workflow):
    # Bước 1: Nhận yêu cầu và lập kế hoạch kiểm tra hợp đồng
    @step()
    async def formulate_plan(self, ctx: Context, ev: StartEvent) -> OutlineEvent:
        query = ev.query  # Ví dụ: "Kiểm tra hợp đồng lao động theo luật Việt Nam"
        await ctx.set("original_query", query)
        await ctx.set("tools", ev.tools)  # Công cụ tra cứu luật

        # Prompt yêu cầu agent ngữ cảnh phân tích loại hợp đồng và các yếu tố cần kiểm tra
        prompt = (
            "Dựa trên yêu cầu '{query}', lập kế hoạch kiểm tra hợp đồng lao động. "
            "Xác định loại hợp đồng (vô thời hạn, xác định thời hạn, thời vụ) và các yếu tố cần kiểm tra "
            "(lương, thời gian làm việc, bảo hiểm, điều kiện chấm dứt, v.v.) theo Bộ luật Lao động 2019 Việt Nam."
        ).format(query=query)

        response = await Settings.llm.acomplete(prompt)
        ctx.write_event_to_stream(ProgressEvent(progress="Kế hoạch kiểm tra:\n" + str(response)))

        return OutlineEvent(outline=str(response))

    # Bước 2: Tạo câu hỏi dựa trên kế hoạch
    @step()
    async def formulate_questions(self, ctx: Context, ev: OutlineEvent) -> QuestionEvent:
        outline = ev.outline
        await ctx.set("outline", outline)

        # Prompt yêu cầu tạo câu hỏi cụ thể để kiểm tra hợp đồng
        prompt = (
            "Dựa trên kế hoạch sau:\n{outline}\n"
            "Tạo danh sách câu hỏi để kiểm tra hợp đồng lao động, ví dụ: "
            "'Lương có đáp ứng mức tối thiểu theo luật không?' hoặc "
            "'Có quy định bảo hiểm xã hội không?'"
        ).format(outline=outline)

        response = await Settings.llm.acomplete(prompt)
        questions = str(response).split("\n")
        questions = [q.strip() for q in questions if q.strip()]

        ctx.write_event_to_stream(
            ProgressEvent(progress="Danh sách câu hỏi:\n" + "\n".join(questions))
        )
        await ctx.set("num_questions", len(questions))

        for question in questions:
            ctx.send_event(QuestionEvent(question=question))

    # Bước 3: Trả lời từng câu hỏi (agent tra cứu pháp luật, logic, rủi ro)
    @step()
    async def answer_question(self, ctx: Context, ev: QuestionEvent) -> AnswerEvent:
        question = ev.question
        if not question or question.isspace():
            ctx.write_event_to_stream(ProgressEvent(progress=f"Bỏ qua câu hỏi rỗng: {question}"))
            return None

        # Khởi tạo agent với công cụ tra cứu luật lao động
        agent = FunctionCallingAgent.from_tools(
            await ctx.get("tools"), verbose=True
        )
        response = await agent.aquery(question)

        # Trích xuất nguồn luật từ phản hồi (nếu có)
        citations = defaultdict(set)
        for node in response.source_nodes:
            metadata = node.node.metadata
            page_number = metadata.get("page_label", "N/A")
            file_name = metadata.get("file_name", "Bộ luật Lao động 2019")
            citations[file_name].add(page_number)

        citation_text = "; ".join(
            f"{file} (trang {', '.join(sorted(pages))})" for file, pages in citations.items()
        )
        final_answer = f"{response}\n\nNguồn: {citation_text if citation_text else 'Không có trích dẫn cụ thể'}"

        ctx.write_event_to_stream(
            ProgressEvent(progress=f"Câu hỏi '{question}' được trả lời: {final_answer}")
        )
        return AnswerEvent(question=question, answer=str(final_answer))

    # Bước 4: Tổng hợp báo cáo phân tích hợp đồng
    @step()
    async def write_report(self, ctx: Context, ev: AnswerEvent) -> ReportEvent:
        num_questions = await ctx.get("num_questions")
        results = ctx.collect_events(ev, [AnswerEvent] * num_questions)
        if results is None:
            return None

        # Lưu trữ tất cả câu hỏi và câu trả lời
        try:
            previous_questions = await ctx.get("previous_questions")
        except:
            previous_questions = []
        previous_questions.extend(results)
        await ctx.set("previous_questions", previous_questions)

        qa_pairs = "\n".join(
            f"<question>{res.question}</question>\n<answer>{res.answer}</answer>"
            for res in previous_questions
        )

        # Prompt tổng hợp báo cáo
        prompt = (
            "Dựa trên kế hoạch:\n{outline}\n"
            "Và các câu hỏi/trả lời:\n{qa_pairs}\n"
            "Tạo báo cáo phân tích hợp đồng lao động, bao gồm: "
            "- Các lỗi pháp lý (nếu có) kèm trích dẫn luật. "
            "- Đề xuất sửa đổi. "
            "- Đánh giá tính hợp pháp và rủi ro."
        ).format(outline=await ctx.get("outline"), qa_pairs=qa_pairs)

        report = await Settings.llm.acomplete(prompt)
        ctx.write_event_to_stream(ProgressEvent(progress="Báo cáo phân tích:\n" + str(report)))

        return ReportEvent(report=str(report))

    # Bước 5: Đánh giá báo cáo, nếu cần thì tạo thêm câu hỏi
    @step()
    async def review_report(self, ctx: Context, ev: ReportEvent) -> Union[StopEvent, QuestionEvent]:
        try:
            num_reviews = await ctx.get("num_reviews")
        except:
            num_reviews = 0
        num_reviews += 1
        await ctx.set("num_reviews", num_reviews)

        report = ev.report
        prompt = (
            "Đánh giá báo cáo sau dựa trên yêu cầu ban đầu '{original_query}':\n{report}\n"
            "Nếu báo cáo đã đầy đủ và chính xác, trả về 'OKAY'. "
            "Nếu cần bổ sung, liệt kê các câu hỏi mới để kiểm tra thêm."
        ).format(original_query=await ctx.get("original_query"), report=report)

        response = await Settings.llm.acomplete(prompt)

        if response == "OKAY" or num_reviews >= 3:
            ctx.write_event_to_stream(ProgressEvent(progress="Báo cáo đã hoàn thiện"))
            return StopEvent(result=report)
        else:
            questions = str(response).split("\n")
            questions = [q.strip() for q in questions if q.strip()]
            await ctx.set("num_questions", len(questions))
            ctx.write_event_to_stream(
                ProgressEvent(progress="Tạo thêm câu hỏi:\n" + "\n".join(questions))
            )
            for question in questions:
                ctx.send_event(QuestionEvent(question=question))

Running step formulate_plan
Step formulate_plan produced event OutlineEvent
Running step formulate_questions
Step formulate_questions produced no event
Running step answer_question
> Running step 7f0c66d8-1d7f-4f50-a3b5-d040a3c347ad. Step input: Dưới đây là danh sách các câu hỏi để kiểm tra hợp đồng lao động:
Added user message to memory: Dưới đây là danh sách các câu hỏi để kiểm tra hợp đồng lao động:
Running step answer_question
> Running step 88597484-8566-4c9b-823e-feeaeff282e3. Step input: 1. Lương:
Added user message to memory: 1. Lương:
Running step answer_question
> Running step 844e058a-06b8-44d4-9ae9-76997c1280d9. Step input: * Mức lương được trả cho nhân viên?
Added user message to memory: * Mức lương được trả cho nhân viên?
Running step answer_question
> Running step 0f4d5cbe-fb62-458e-b146-9003bd42b5c1. Step input: * Kỳ hạn trả lương và phương thức thanh toán?
Added user message to memory: * Kỳ hạn trả lương và phương thức thanh toán?
=== Calling Function ===
Calling funct

In [ ]:
# Ví dụ chạy workflow
async def main():
    workflow = LaborContractReviewWorkflow(timeout=3000, verbose=True)
    result = await workflow.run(
        query="Kiểm tra hợp đồng lao động thời hạn 1 năm, lương 8 triệu/tháng, thử việc 3 tháng theo luật Việt Nam",
        tools=[law_tool]  # Thêm công cụ tra cứu luật tại đây
    )
    print(result)

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())